In [ ]:
# -*- coding: utf-8 -*-
"""Docking_in_Colab.ipynb"""

# Install dependencies
print("Installing dependencies...")
!apt-get install -y openbabel
!pip install --no-build-isolation pandas==2.2.2 rdkit biopandas py3Dmol vina
print("✅ Dependencies installed")

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdmolfiles
from vina import Vina
import py3Dmol
import os

# Clean up existing files to avoid overwrite errors
print("Cleaning up existing files...")
for file in ["drug.pdb", "drug.pdbqt", "protein.pdbqt", "docked_out.pdbqt"]:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removed {file}")

# Function to parse affinity scores from PDBQT file
def parse_pdbqt_affinities(pdbqt_file):
    affinities = []
    try:
        with open(pdbqt_file, 'r') as f:
            for line in f:
                if line.startswith("REMARK VINA RESULT"):
                    parts = line.split()
                    if len(parts) > 3:
                        try:
                            affinity = float(parts[3])
                            affinities.append(affinity)
                        except ValueError:
                            continue
        return affinities
    except FileNotFoundError:
        print(f"Error: {pdbqt_file} not found")
        return []
    except Exception as e:
        print(f"Error parsing {pdbqt_file}: {e}")
        return []

# SMILES string of the drug
smiles = "O)(=N(=N=C(=C=C(=N)(=C=C=C=C=C=C=C=C=NCN)N=C=C=C=C"

# Convert to molecule
print("Generating 3D structure for ligand...")
mol = Chem.MolFromSmiles(smiles)
if mol is None:
    raise ValueError("Failed to parse SMILES string")
mol = Chem.AddHs(mol)

# Generate 3D conformer
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# Save as PDB
rdmolfiles.MolToPDBFile(mol, "drug.pdb")
print("✅ 3D structure saved as 'drug.pdb'")

# Convert ligand PDB to PDBQT
print("Converting ligand PDB to PDBQT...")
os.system("obabel drug.pdb -O drug.pdbqt -xh")
if not os.path.exists("drug.pdbqt"):
    raise FileNotFoundError("Failed to generate drug.pdbqt")
print("✅ Converted 'drug.pdb' to 'drug.pdbqt'")

# Convert protein PDB to PDBQ
protein_pdb = "/content/test_db3c7_unrelaxed_rank_001_alphafold2_ptm_model_4_seed_000.pdb"  # Adjust to match your ColabFold PDB filename
protein_pdbqt = "protein.pdbqt"
print("Converting protein PDB to PDBQT...")
os.system(f"obabel {protein_pdb} -O {protein_pdbqt} -xr")
if not os.path.exists(protein_pdbqt):
    raise FileNotFoundError("Failed to generate protein.pdbqt")
print("✅ Converted protein PDB to 'protein.pdbqt'")

# Initialize Vina
print("Setting up AutoDock Vina...")
v = Vina(sf_name='vina')

# Load receptor and ligand
v.set_receptor(protein_pdbqt)
v.set_ligand_from_file("drug.pdbqt")

# Define docking box
v.compute_vina_maps(center=[0, 0, 0], box_size=[25, 25, 25])

# Perform docking
print("Running docking...")
try:
    v.dock(exhaustiveness=8, n_poses=5)
    v.write_poses("docked_out.pdbqt", n_poses=5, overwrite=True)  # Allow overwriting
    print("✅ Docking completed, poses saved as 'docked_out.pdbqt'")
except Exception as e:
    print(f"Error during docking: {e}")
    raise

# Binding affinity
print("Retrieving affinity scores...")
scores = v.poses()
if scores:
    try:
        affinities = [pose[1] for pose in scores]
        print("Affinity scores (kcal/mol) from v.poses():", affinities)
    except (IndexError, TypeError) as e:
        print(f"Error with v.poses(): {e}. Falling back to parsing docked_out.pdbqt...")
        affinities = parse_pdbqt_affinities("docked_out.pdbqt")
        print("Affinity scores (kcal/mol) from docked_out.pdbqt:", affinities)
else:
    print("No poses returned by v.poses(). Parsing docked_out.pdbqt...")
    affinities = parse_pdbqt_affinities("docked_out.pdbqt")
    print("Affinity scores (kcal/mol) from docked_out.pdbqt:", affinities)

# Visualize results
print("Loading visualization...")
view = py3Dmol.view(width=600, height=400)
try:
    view.addModel(open(protein_pdb, 'r').read(), 'pdb')  # Use PDB for visualization
    view.setStyle({'cartoon': {'color': 'spectrum'}})
    view.addModel(open('docked_out.pdbqt', 'r').read(), 'pdb')
    view.setStyle({'stick': {}})
    view.zoomTo()
    view.show()
    print("✅ Visualization loaded")
except FileNotFoundError as e:
    print(f"Error: Could not find file for visualization: {e}")
except Exception as e:
    print(f"Error in visualization: {e}")

Installing dependencies...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7
The following NEW packages will be installed:
  libinchi1 libmaeparser1 libopenbabel7 openbabel
0 upgraded, 4 newly installed, 0 to remove and 38 not upgraded.
Need to get 3,903 kB of archives.
After this operation, 16.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinchi1 amd64 1.03+dfsg-4 [455 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmaeparser1 amd64 1.2.4-1build1 [88.2 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenbabel7 amd64 3.1.1+dfsg-6ubuntu5 [3,231 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openbabel amd64 3.1.1+dfsg-6ubuntu5 [128 kB]
Fetched 3,903 kB in 0s (11.5 MB/s)
Selecting previously unselected package libinchi1.
(Reading database

[10:02:25] SMILES Parse Error: extra close parentheses while parsing: O)(=N(=N=C(=C=C(=N)(=C=C=C=C=C=C=C=C=NCN)N=C=C=C=C
[10:02:25] SMILES Parse Error: check for mistakes around position 2:
[10:02:25] O)(=N(=N=C(=C=C(=N)(=C=C=C=C=C=C=C=C=NCN)
[10:02:25] ~^
[10:02:25] SMILES Parse Error: Failed parsing SMILES 'O)(=N(=N=C(=C=C(=N)(=C=C=C=C=C=C=C=C=NCN)N=C=C=C=C' for input: 'O)(=N(=N=C(=C=C(=N)(=C=C=C=C=C=C=C=C=NCN)N=C=C=C=C'


ValueError: Failed to parse SMILES string